<a href="https://colab.research.google.com/github/shubham-k01/D2K-Text-Summarisation/blob/main/D2K_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Extracting the data**

In [75]:
with open('/content/PS3_Dataset.txt') as f:
  l = f.read()
  # l = l.replace(' ',',')
  print(l)

https://pubs.acs.org/doi/10.1021/acs.chemrev.1c00107

https://downloads.hindawi.com/journals/cin/2022/5624475.pdf

https://pubs.acs.org/doi/pdf/10.1021/acssynbio.2c00015

https://downloads.hindawi.com/journals/cin/2022/5624475.pdf

https://link.springer.com/article/10.1007/s11023-021-09573-8

https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0237749&type=printable

https://www.mdpi.com/2079-4991/12/15/2646

https://pubs.acs.org/doi/10.1021/acs.chemrev.0c00004

https://www.hindawi.com/journals/jhe/2022/4653923/

https://www.mdpi.com/1660-4601/19/22/15027

https://www.nature.com/articles/s41598-021-84637-4

https://onlinelibrary.wiley.com/doi/10.1111/1754-9485.13274

https://www.mdpi.com/1424-8220/22/23/9148

https://www.frontiersin.org/articles/10.3389/fnagi.2021.633752/full

https://pubs.asahq.org/anesthesiology/article/132/2/379/108833/Artificial-Intelligence-in-AnesthesiologyCurrent

https://www.mdpi.com/2072-6694/14/6/1524

https://translational-medicine.biomedc

# **Converting PDF to text**

In [ ]:
!pip install PyPDF2

In [77]:
from PyPDF2 import PdfReader
 
# creating a pdf reader object
reader = PdfReader('/content/drive/MyDrive/D2K_Data/sample0.pdf')
 
# printing number of pages in pdf file
print(len(reader.pages))
 
# getting a specific page from the pdf file
page = reader.pages[0]
 
# extracting text from page
text = page.extract_text()
print(text)

21
Health Policy and Technology  10 (2021) 209–229  
Contents  lists available  at ScienceDirect  
Health  Policy  and Technology  
journal  homepage:  www.elsevier.com/locate/hlpt  
Literature  Review  
Telemedicine  and health  policy:  A systematic  review  
Clemens  Scott Kruse  ∗, Kelly Williams  1 , John Bohls  1 , Waleed  Shamsi  1 
School of Health Administration,  Texas State University,  San Marcos, TX, United States 
a r t i c l e i n f o 
Article history: 
Available  online 29 October 2020 
Keywords:  
Health 
Policy 
Telemedicine  
Access 
Health policy a b s t r a c t 
Background:  Telemedicine  diagnoses  and treats patients  remotely  via telecommunications  technology  all 
over the world. Telemedicine  becomes  more prevalent  as providers  recognize  the beneﬁts,  patients  re- 
ceive increased  access and payers see the reduction  in cost of care. 
Objective:  Telemedicine  studies have shown success in limiting  geographical  constraints,  time spent, and 
costs in

In [78]:
folder = '/content/drive/MyDrive/D2K_Data'

In [ ]:
data = []
for i in range(0,18):
  reader = PdfReader(f'{folder}/sample{i}.pdf')
  number_of_pages = len(reader.pages)
  li = ''
  for j in range(number_of_pages):
    page = reader.pages[j]
    page_content = page.extract_text()
    li += page_content
  data.append({f'sample{i}':li})

In [ ]:
len(data)

###**Text Summarization**###

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
doc = nlp(data[0]['sample0'])

In [ ]:
tokens = [token.text for token in doc]
print(tokens)

In [ ]:
punctuation = punctuation + '\n'
punctuation

In [ ]:
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
                
print(word_frequencies)

In [ ]:
max_frequency = max(word_frequencies.values())
max_frequency

In [ ]:
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

print(word_frequencies)

In [ ]:
sentence_tokens = [sent for sent in doc.sents]
print(sentence_tokens)

In [ ]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.text.lower()]
                
sentence_scores

In [ ]:
from heapq import nlargest

In [ ]:
select_length = int(len(sentence_tokens)*0.3)
select_length

## Health care Domain keywords tokenzation

In [ ]:
keywords = ['Artificial intelligence in healthcare', 'Big data analytics in healthcare',  'Clinical decision support systems', 'Electronic health records', 'Health information technology',             'Health insurance', 'Health policy', 'Health promotion', 'Health services research', 'Healthcare financing',             'Healthcare management', 'Healthcare quality improvement', 'Healthcare regulation', 'Healthcare technology',             'Hospital administration', 'Medical ethics', 'Medical informatics', 'Medical imaging', 'Medical research',             'Mental health', 'Nutrition and dietetics', 'Occupational therapy', 'Patient safety', 'Pediatrics', 'Pharmacy',             'Physical therapy', 'Public health', 'Quality of life', 'Radiology', 'Rehabilitation', 'Reproductive health',             'Social work', 'Telemedicine', 'Vaccination', "Women's health", 'Aging and geriatrics', 'Alternative medicine',             'Cancer research', 'Cardiovascular diseases', 'Chronic diseases', 'Complementary medicine', 'Dental health',             'Dermatology', 'Endocrinology', 'Epidemiology', 'Gastroenterology', 'Genetics', 'Infectious diseases', 'Neurology',             'Respiratory diseases']


In [ ]:
sentence_scores

In [ ]:
summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
summary

In [ ]:
t=set(summary)
t

## **NLTK**

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize



In [ ]:
nltk.download("stopwords")
stop_words = stopwords.words('english')

In [ ]:
punctuation = punctuation + '\n'
punctuation

In [ ]:
word_frequencies = {}
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
                
print(word_frequencies)

In [ ]:
max_frequency = max(word_frequencies.values())
max_frequency

In [ ]:
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequency

In [ ]:
print(word_frequencies)

In [ ]:
sent_token = sent_tokenize(data[0]['sample0'])
sent_token

In [ ]:
sentence_scores = {}
for sent in sent_token:
    sentence = sent.split(" ")
    for word in sentence:        
        if word.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():
                sentence_scores[sent] = word_frequencies[word.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.lower()]

In [ ]:
len(sent_token)

In [ ]:
sentence_scores

In [ ]:
from heapq import nlargest
select_length = int(len(sent_token)*0.3)
select_length

In [ ]:
summary = nlargest(select_length, sentence_scores, key = sentence_scores.get)
final_summary = [word for word in summary]
summary = ' '.join(final_summary)
summary = summary.replace('\n',' ').replace('\r',' ')
summary

In [ ]:
!pip install rouge


In [ ]:
from rouge import Rouge
rouge = Rouge()

def evaluate_summarizer(reference, summary):
    """
    Evaluates a text summarizer using the Rouge algorithm.

    Args:
    - reference (str): the reference text to summarize
    - summary (str): the summary generated by the text summarizer

    Returns:
    - rouge_scores (dict): a dictionary of Rouge scores (Rouge-1, Rouge-2, Rouge-L) for the summary
    """
    scores = rouge.get_scores(summary, reference)
    rouge_scores = {
        "rouge-1": scores[0]["rouge-1"],
        "rouge-2": scores[0]["rouge-2"],
        "rouge-l": scores[0]["rouge-l"],
    }
    return rouge_scores

In [ ]:
evaluate_summarizer(data[0]['sample0'],summary)

In [ ]:
import spacy
from collections import Counter

nlp = spacy.load('en_core_web_sm')

def summarize_section(section_text):
    """
    Given the text of a section, generate a summary using NLP and machine learning techniques.
    """
    # Remove non-alphanumeric characters and convert to lowercase
    section_text = ''.join(c for c in section_text if c.isalnum() or c.isspace())
    section_text = section_text.lower()
    
    # Tokenize and count the frequency of each word
    doc = nlp(section_text)
    word_counts = Counter(token.text for token in doc if not token.is_stop and not token.is_punct)
    
    # Extract the top 5 most frequent words and return them as the summary
    summary = [word for word, count in word_counts.most_common(5)]
    summary = " ".join(summary)
    
    return summary


def summarize_paper(paper_text):
    """
    Given the text of a research paper, generate section-wise summaries using NLP and machine learning techniques.
    """
    # Split the paper into sections
    sections = paper_text.split("\n\n")
    section_names = ["Abstract", "Methodology", "Evaluation", "Conclusion", "Discussion"]
    
    # Generate a summary for each section
    summaries = {}
    for i in range(len(sections)):
        section_name = section_names[i] if i < len(section_names) else "Additional section"
        section_text = sections[i]
        section_summary = summarize_section(section_text)
        summaries[section_name] = section_summary
    
    return summaries

# Example usage
paper_text = """
Abstract: This paper presents a new method for sentiment analysis using deep learning. We propose a novel architecture that combines convolutional and recurrent neural networks to capture both local and global context. Our experiments on a benchmark dataset show that our method outperforms state-of-the-art approaches in terms of accuracy and efficiency.

Methodology: We collected a corpus of 10,000 movie reviews and preprocessed the text using standard techniques such as tokenization, stopword removal, and stemming. We then split the corpus into training and testing sets and trained our sentiment analysis model using TensorFlow. The model consists of 3 convolutional layers and 2 LSTM layers, with a final softmax layer for classification.

Evaluation: We evaluated our model on the popular IMDB sentiment analysis dataset and achieved an accuracy of 90%, outperforming the previous state-of-the-art method which had an accuracy of 87%. We also conducted ablation studies to investigate the contribution of each component in our model and found that both convolutional and recurrent layers are important for achieving high accuracy.

Conclusion: In conclusion, we presented a new method for sentiment analysis using a combination of convolutional and recurrent neural networks. Our experiments on a benchmark dataset showed that our method outperforms state-of-the-art approaches in terms of accuracy and efficiency.

Discussion: We believe that our method has the potential to be applied to other natural language processing tasks and lead to further improvements in the field. However, there are still some limitations to our approach, such as the need for large amounts of labeled data and the relatively high computational requirements of deep learning models.
"""
summaries = summarize_paper(paper_text)

print("Abstract summary:", summaries["Abstract"])
print("Methodology summary:", summaries["Methodology"])
print("Evaluation summary:", summaries["Evaluation"])
print("Conclusion summary:", summaries["Conclusion"])
print("Discussion summary:", summaries["Discussion"])